In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/bigquery-ml/bqml-vertexai-model-registry.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/bigquery-ml/bqml-vertexai-model-registry.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/bigquery-ml/bqml-vertexai-model-registry.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>                                                                                               
</table>

# E2E ML on GCP: Deploy BiqQuery ML Model on Vertex AI Model Registry and Make Predictions

## Overview

This tutorial demonstrates how to train a model with BigQuery Machine Learning and upload it on Vertex AI model registry, then make batch predictions.

### Dataset

The dataset used for this tutorial is the Penguins dataset from [BigQuery public datasets](https://cloud.google.com/bigquery/public-data). This version of the dataset is used to predict the species of penguins from the available features like culmen-length, flipper-depth etc.


### Objective

In this tutorial, you learn how to use `Vertex AI Model Registry` with `BigQuery Machine Learning` and make batch predictions:

This tutorial uses the following Google Cloud ML services and resources:

- `Vertex AI Model Registry`
- `Vertex AI Model` resources 
- `Vertex AI Endpoint` resources
- `Vertex AI Prediction`
- `BigQuery Machine Learning`


The steps performed include:

- Train a model with `BQML`
- Upload the model to `Vertex AI Model Registry` 
- Create a `Vertex AI Endpoint` resource
- Deploy the `Model` resource to the `Endpoint` resource
- Make `prediction` requests to the model endpoint
- Run `batch prediction` job on the `Model` resource 

### Costs 

This tutorial uses billable components of Google Cloud:

* Vertex AI
* BigQuery Machine Learning 

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing) and [BigQuery pricing](https://cloud.google.com/bigquery/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

### Set up your local development environment

**If you are using Colab or Vertex AI Workbench Notebooks**, your environment already meets
all the requirements to run this notebook. You can skip this step.

**Otherwise**, make sure your environment meets this notebook's requirements.
You need the following:

* The Google Cloud SDK
* Git
* Python 3
* virtualenv
* Jupyter notebook running in a virtual environment with Python 3

The Google Cloud guide to [Setting up a Python development
environment](https://cloud.google.com/python/setup) and the [Jupyter
installation guide](https://jupyter.org/install) provide detailed instructions
for meeting these requirements. The following steps provide a condensed set of
instructions:

1. [Install and initialize the Cloud SDK.](https://cloud.google.com/sdk/docs/)

1. [Install Python 3.](https://cloud.google.com/python/setup#installing_python)

1. [Install
   virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)
   and create a virtual environment that uses Python 3. Activate the virtual environment.

1. To install Jupyter, run `pip3 install jupyter` on the
command-line in a terminal shell.

1. To launch Jupyter, run `jupyter notebook` on the command-line in a terminal shell.

1. Open this notebook in the Jupyter Notebook Dashboard.

### Install additional packages

Install the following packages required to execute this notebook. 

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install --upgrade google-cloud-aiplatform {USER_FLAG} -q
! pip3 install --upgrade google-cloud-bigquery {USER_FLAG} -q

     |████████████████████████████████| 2.1 MB 5.6 MB/s 
     |████████████████████████████████| 106 kB 67.9 MB/s 
     |████████████████████████████████| 1.0 MB 53.3 MB/s 
     |████████████████████████████████| 46 kB 3.9 MB/s 
     |████████████████████████████████| 231 kB 69.9 MB/s 
     |████████████████████████████████| 114 kB 59.4 MB/s 
     |████████████████████████████████| 107 kB 63.0 MB/s 
     |████████████████████████████████| 107 kB 62.9 MB/s 
     |████████████████████████████████| 107 kB 50.9 MB/s 
     |████████████████████████████████| 106 kB 61.5 MB/s 
     |████████████████████████████████| 106 kB 61.5 MB/s 
     |████████████████████████████████| 106 kB 61.8 MB/s 
     |████████████████████████████████| 106 kB 63.3 MB/s 
     |████████████████████████████████| 105 kB 61.2 MB/s 
     |████████████████████████████████| 105 kB 54.2 MB/s 
     |████████████████████████████████| 105 kB 38.4 MB/s 
     |████████████████████████████████| 105 kB 64.0 MB/s 
     |███████████

### Restart the kernel

After you install the additional packages, you need to restart the notebook kernel so it can find the packages.

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Before you begin

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

1. [Enable the Vertex AI and BigQuery APIs](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,bigquery). 

1. If you are running this notebook locally, you will need to install the [Cloud SDK](https://cloud.google.com/sdk).

1. Enter your project ID in the cell below. Then run the cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [ ]:
import os

PROJECT_ID = ""

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

Otherwise, set your project ID here.

In [ ]:
PROJECT_ID = "bqml-vertexai-356715"  # @param {type:"string"}

In [ ]:
! gcloud config set project $PROJECT_ID

Updated property [core/project].


In [ ]:
! gcloud config get project

bqml-vertexai-356715


#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex AI. We recommend that you choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You may not use a multi-regional bucket for training with Vertex AI. Not all regions provide support for all Vertex AI services.

Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

### Authenticate your Google Cloud account

**If you are using Vertex AI Workbench Notebooks**, your environment is already
authenticated. Skip this step.

**If you are using Colab**, run the cell below and follow the instructions
when prompted to authenticate your account via oAuth.

**Otherwise**, follow these steps:

1. In the Cloud Console, go to the [**Create service account key**
   page](https://console.cloud.google.com/apis/credentials/serviceaccountkey).

2. Click **Create service account**.

3. In the **Service account name** field, enter a name, and
   click **Create**.

4. In the **Grant this service account access to project** section, click the **Role** drop-down list. Type "Vertex AI"
into the filter box, and select
   **Vertex AI Administrator**. Type "Storage Object Admin" into the filter box, and select **Storage Object Admin**.

5. Click *Create*. A JSON file that contains your key downloads to your
local environment.

6. Enter the path to your service account key as the
`GOOGLE_APPLICATION_CREDENTIALS` variable in the cell below and run the cell.

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### Import libraries

In [ ]:
import google.cloud.aiplatform as aiplatform
from google.cloud import bigquery

### Initialize Vertex AI and Big Query SDKs for Python

Initialize the Vertex AI and Big Query SDKs for Python for your project and corresponding bucket.

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION)

In [ ]:
bqclient = bigquery.Client(project=PROJECT_ID)

## BigQuery ML introduction

BigQuery ML (BQML) provides the capability to train ML tabular models, such as classification and regression, in BigQuery using SQL syntax.

Learn more about [BigQuery ML documentation](https://cloud.google.com/bigquery-ml/docs).

In [ ]:
# Define BigQuery table to be used for training

BQ_TABLE = "bigquery-public-data.ml_datasets.penguins"

### Create BQ dataset resource

First, you create an empty dataset resource in your project.

In [ ]:
BQ_DATASET_NAME = "penguins"
DATASET_QUERY = f"""CREATE SCHEMA {BQ_DATASET_NAME}"""

job = bqclient.query(DATASET_QUERY)

## Train BQML model and upload it to Vertex AI Model Registry

Next, you create and train a BQML tabular regression model from the public dataset penguins and store the model in your project `Vertex AI Model Registry` using the `CREATE MODEL` statement. The model configuration is specified in the `OPTIONS` statement as follows:

- `model_type`: The type and archictecture of tabular model to train, e.g., LOGISTIC_REG.

- `labels`: The column which are the labels.

- `model_registry`: To register a BigQuery ML model to Vertex AI Model Registry, you must use `model_registry="vertex_ai"`.

Learn more about the [CREATE MODEL statement](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create).

Learn more about [Managing BigQuery ML models in the Vertex AI Model Registry](https://cloud.google.com/bigquery-ml/docs/managing-models-vertex).

In [ ]:
# Write the query to create Big Query ML model

MODEL_NAME = "penguins-lr"
MODEL_QUERY = f"""
CREATE OR REPLACE MODEL `{BQ_DATASET_NAME}.{MODEL_NAME}`
OPTIONS(
    model_type='LOGISTIC_REG',
    labels = ['species'],
    model_registry='vertex_ai'
    )
AS
SELECT *
FROM `{BQ_TABLE}`
"""

In [ ]:
# Run the model creation query using Big Query client

job = bqclient.query(MODEL_QUERY)
print(f"Job state: {job.state}\nJob Error:{job.errors}")

Job state: RUNNING
Job Error:None


In [ ]:
while job.running():
    from time import sleep

    sleep(30)
    print("Running ...")
print(job.errors, job.state)

Running ...
Running ...
Running ...
[{'reason': 'invalidQuery', 'location': 'query', 'message': 'The input data has NULL values in one or more columns: culmen_length_mm, culmen_depth_mm, flipper_length_mm, body_mass_g, sex. BQML automatically handles null values (See https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-auto-preprocessing#imputation). If null values represent a special value in the data, replace them with the desired value before training and then retry.'}] DONE


### Find the model in the Vertex Model Registry

You can use the `Vertex AI Model list()` method with a filter query to find the automatically registered model.

In [ ]:
models = aiplatform.Model.list(filter=f"display_name={MODEL_NAME}")
model = models[0]

print(model.gca_resource)

name: "projects/704725089108/locations/us-central1/models/penguins-lr"
display_name: "penguins-lr"
supported_deployment_resources_types: DEDICATED_RESOURCES
create_time {
  seconds: 1658430788
  nanos: 441880000
}
update_time {
  seconds: 1658430788
  nanos: 441880000
}
deployed_models {
  endpoint: "projects/704725089108/locations/us-central1/endpoints/1198019073531707392"
  deployed_model_id: "3335518056439873536"
}
etag: "AMEw9yOT8QWWptSveXHhIiAyOicOBeGKJI_iFIJjobGaLVZ5P7PhmL1U6axG-FtWfb_L"
version_id: "1"
version_aliases: "default"
version_create_time {
  seconds: 1658430788
  nanos: 441880000
}
version_update_time {
  seconds: 1658430788
  nanos: 441880000
}



## Deploy the model to an endpoint

You must deploy a model to an `endpoint` before that model can be used to serve online predictions; deploying a model associates physical resources with the model so it can serve online predictions with low latency. 

Learn more about [Deploy a model using the Vertex AI API](https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api#aiplatform_deploy_model_custom_trained_model_sample-python)


### Creat an endpoint

If you are deploying a model to an existing endpoint, you can skip this cell.

In [ ]:
ENDPOINT_DISPLAY_NAME = "bqml-lr-model-endpoint"

endpoint = aiplatform.Endpoint.create(
    display_name=ENDPOINT_DISPLAY_NAME,
    project=PROJECT_ID,
    location=REGION,
)

print(endpoint.display_name)
print(endpoint.resource_name)

Creating Endpoint


INFO:google.cloud.aiplatform.models:Creating Endpoint


Create Endpoint backing LRO: projects/704725089108/locations/us-central1/endpoints/1198019073531707392/operations/5570802724439588864


INFO:google.cloud.aiplatform.models:Create Endpoint backing LRO: projects/704725089108/locations/us-central1/endpoints/1198019073531707392/operations/5570802724439588864


Endpoint created. Resource name: projects/704725089108/locations/us-central1/endpoints/1198019073531707392


...

To use this Endpoint in another session:


INFO:google.cloud.aiplatform.models:To use this Endpoint in another session:


endpoint = aiplatform.Endpoint('projects/704725089108/locations/us-central1/endpoints/1198019073531707392')


INFO:google.cloud.aiplatform.models:endpoint = aiplatform.Endpoint('projects/704725089108/locations/us-central1/endpoints/1198019073531707392')


bqml-lr-model-endpoint
projects/704725089108/locations/us-central1/endpoints/1198019073531707392


If you want to use an existing endpoint, enter the endpoint ID below.

In [ ]:
# get an existing endpoint
ENDPOINT_ID = "1198019073531707392"  # @param {type:"string"}
endpoint = aiplatform.Endpoint(ENDPOINT_ID)

### Deploy the model to the endpoint

In [ ]:
DEPLOYED_NAME = "bqml-lr-penguins"

model.deploy(endpoint=endpoint, deployed_model_display_name=DEPLOYED_NAME)

print(model.display_name)
print(model.resource_name)

Deploying model to Endpoint : projects/704725089108/locations/us-central1/endpoints/1198019073531707392


INFO:google.cloud.aiplatform.models:Deploying model to Endpoint : projects/704725089108/locations/us-central1/endpoints/1198019073531707392


Using default machine_type: n1-standard-2


INFO:google.cloud.aiplatform.models:Using default machine_type: n1-standard-2


Deploy Endpoint model backing LRO: projects/704725089108/locations/us-central1/endpoints/1198019073531707392/operations/1177541287939670016


INFO:google.cloud.aiplatform.models:Deploy Endpoint model backing LRO: projects/704725089108/locations/us-central1/endpoints/1198019073531707392/operations/1177541287939670016


Endpoint model deployed. Resource name: projects/704725089108/locations/us-central1/endpoints/1198019073531707392


INFO:google.cloud.aiplatform.models:Endpoint model deployed. Resource name: projects/704725089108/locations/us-central1/endpoints/1198019073531707392


penguins-lr
projects/704725089108/locations/us-central1/models/penguins-lr


## Send prediction request to the endpoint

Now that your Model resource is deployed to an `Endpoint` resource, you can do online predictions by sending prediction requests to the `Endpoint` resource.

Learn more about [Get online predictions from custom-trained models ](https://cloud.google.com/vertex-ai/docs/predictions/online-predictions-custom-models)

In [ ]:
instance = {
    "island": "Dream",
    "culmen_length_mm": 36.6,
    "culmen_depth_mm": 18.4,
    "flipper_length_mm": 184.0,
    "body_mass_g": 3475.0,
    "sex": "FEMALE",
}

endpoint.predict([instance])

Prediction(predictions=[{'predicted_species': ['Adelie Penguin (Pygoscelis adeliae)'], 'species_values': ['Gentoo penguin (Pygoscelis papua)', 'Adelie Penguin (Pygoscelis adeliae)', 'Chinstrap penguin (Pygoscelis antarctica)'], 'species_probs': [8.708736467696576e-06, 0.9958568976496748, 0.004134393613857475]}], deployed_model_id='3335518056439873536', model_version_id='', model_resource_name='projects/704725089108/locations/us-central1/models/penguins-lr', explanations=None)

## Batch Prediction on the BQML model

Here you request batch predictions directly from the model resource; you don't need to deploy the model to an endpoint. For data types that support both batch and online predictions, use batch predictions when you don't require an immediate response and want to process accumulated data by using a single request.

Learn more abount [The ML.PREDICT function](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-predict)

In [ ]:
sql_ml_predict = f"""SELECT * FROM ML.PREDICT(MODEL `{PROJECT_ID}.{BQ_DATASET_NAME}.{MODEL_NAME}`, 
(SELECT
      *
    FROM
      `{BQ_TABLE}` LIMIT 10))"""

job = bqclient.query(sql_ml_predict)
prediction_result = job.result().to_arrow().to_pandas()

In [ ]:
prediction_result

,predicted_species,predicted_species_probs,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
0,Chinstrap penguin (Pygoscelis antarctica),[{'label': 'Chinstrap penguin (Pygoscelis anta...,Chinstrap penguin (Pygoscelis antarctica),Dream,50.5,18.4,200.0,3400.0,FEMALE
1,Adelie Penguin (Pygoscelis adeliae),[{'label': 'Adelie Penguin (Pygoscelis adeliae...,Adelie Penguin (Pygoscelis adeliae),Dream,39.8,19.1,184.0,4650.0,MALE
2,Chinstrap penguin (Pygoscelis antarctica),[{'label': 'Chinstrap penguin (Pygoscelis anta...,Chinstrap penguin (Pygoscelis antarctica),Dream,46.9,16.6,192.0,2700.0,FEMALE
3,Chinstrap penguin (Pygoscelis antarctica),[{'label': 'Chinstrap penguin (Pygoscelis anta...,Chinstrap penguin (Pygoscelis antarctica),Dream,49.5,19.0,200.0,3800.0,MALE
4,Chinstrap penguin (Pygoscelis antarctica),[{'label': 'Chinstrap penguin (Pygoscelis anta...,Chinstrap penguin (Pygoscelis antarctica),Dream,46.5,17.9,192.0,3500.0,FEMALE
5,Adelie Penguin (Pygoscelis adeliae),[{'label': 'Adelie Penguin (Pygoscelis adeliae...,Adelie Penguin (Pygoscelis adeliae),Dream,43.2,18.5,192.0,4100.0,MALE
6,Adelie Penguin (Pygoscelis adeliae),[{'label': 'Adelie Penguin (Pygoscelis adeliae...,Adelie Penguin (Pygoscelis adeliae),Dream,40.9,18.9,184.0,3900.0,MALE
7,Adelie Penguin (Pygoscelis adeliae),[{'label': 'Adelie Penguin (Pygoscelis adeliae...,Adelie Penguin (Pygoscelis adeliae),Dream,37.3,16.8,192.0,3000.0,FEMALE
8,Adelie Penguin (Pygoscelis adeliae),[{'label': 'Adelie Penguin (Pygoscelis adeliae...,Adelie Penguin (Pygoscelis adeliae),Dream,36.6,18.4,184.0,3475.0,FEMALE
9,Adelie Penguin (Pygoscelis adeliae),[{'label': 'Adelie Penguin (Pygoscelis adeliae...,Adelie Penguin (Pygoscelis adeliae),Dream,40.2,20.1,200.0,3975.0,MALE


## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

{TODO: Include commands to delete individual resources below}

In [ ]:
# Delete the endpoint using the Vertex endpoint object
endpoint.undeploy_all()
endpoint.delete()

# Delete the model using the Vertex model object
try:
    model.delete()
except Exception as e:
    print(e)

# Delete the created BigQuery dataset
! bq rm -r -f $PROJECT_ID:$BQ_DATASET_NAME